# Fichero de permisos

## Origen
Fichero obtenido de la DGT
## Tratamiento
El fichero contiene la cantidad de permisos emitidos por año y por tramo de edad.
Desglosamos los permisos del tipo "A,B,D" en diferentes líneas para contabilizarlos por separado
Acumulamos en los sucesivos años los datos del primer registro que corresponde a los anteriores al 2001

In [19]:
import csv
import pandas as pd
import numpy as np

In [94]:
permisos=[]
acum = {}
i=0
with open ('data/permisosMadrid.csv',encoding="ISO-8859-1") as permisosIn:
        reader = csv.DictReader(permisosIn)
        for row in reader:
            i += 1
            for permiso in row['GRUPO PERMISOS'].split(','):
                if not '+E' in permiso and len(permiso.strip())>0: #Algunos terminan en ',' y generan permiso vacío
                    row['GRUPO PERMISOS']=permiso
                    anyo = row['ANYO_EXPEDICION']
                    total = int(row['Total'])
                    sexo = row['SEXO']

                    if not sexo in acum:
                        acum[sexo]={}
                        #print ('acum {}'.format(sexo))
                    if not anyo in acum[sexo]:
                        acum[sexo][anyo]={}
                        #print ('acum {} {}'.format(sexo, anyo))
                    if not permiso in acum[sexo][anyo]:
                        acum[sexo][anyo][permiso]=0

                    acum [sexo][anyo][permiso] += total
                        #print ('--> total {} acum {}'.format(total, acum[sexo][anyo][permiso]))

                    permisos.append(row)
permisosIn.close()


In [97]:
for sexo in [*acum]
    for anyo in [*acum[sexo]]
        for permiso in [*acum[sexo][anyo]]
            if anyo == '2000':
                tot = acum [sexo][anyo][permiso]
            else:
                acum [sexo][anyo][permiso] += tot
                tot = acum [sexo][anyo][permiso]

['M', 'V']

In [98]:
[*acum['M']]

['2000',
 '2001',
 '2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019']

In [72]:
df = pd.DataFrame(permisos)
df [['HASTA 18', 'DE 19 A 24', 'DE 25 A 34', 'DE 35 A 44', 'DE 45 A 54',
       'DE 55 A 64', 'DE 65 A 74', 'MÁS DE 74', 'Total']] = \
       df [['HASTA 18', 'DE 19 A 24', 'DE 25 A 34', 'DE 35 A 44', 'DE 45 A 54', \
       'DE 55 A 64', 'DE 65 A 74', 'MÁS DE 74', 'Total']].apply(pd.to_numeric)



In [73]:

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1751 entries, 0 to 1750
Data columns (total 14 columns):
FECHA CENSO        1751 non-null object
ANYO_EXPEDICION    1751 non-null object
MUNICIPIO          1751 non-null object
SEXO               1751 non-null object
GRUPO PERMISOS     1751 non-null object
HASTA 18           1751 non-null int64
DE 19 A 24         1751 non-null float64
DE 25 A 34         1751 non-null float64
DE 35 A 44         1751 non-null float64
DE 45 A 54         1751 non-null float64
DE 55 A 64         1751 non-null float64
DE 65 A 74         1751 non-null float64
MÁS DE 74          1751 non-null float64
Total              1751 non-null int64
dtypes: float64(7), int64(2), object(5)
memory usage: 191.6+ KB


In [74]:
dfA = df.groupby (['SEXO', 'GRUPO PERMISOS', 'ANYO_EXPEDICION']) 
dfA

In [80]:
for k, datos in dfA:
    sexo, lic, anyo = k
    if sexo == 'M' and lic == 'A' and anyo == '2000':
        print ('sexo {} lic {} año {}'.format(sexo,lic,anyo))
        #print (datos)
        for indice_fila, fila in datos.iterrows():

            print("Indice {} Total{}".format (indice_fila, fila['Total']))
    
    """
    for dato in datos:
        if float(datos['dato']) > 0:
            print (dato)
"""

sexo M lic A año 2000
Indice 0 Total25
Indice 34 Total11
Indice 35 Total11


In [36]:
dfA

HASTA 18  DE 19 A 24  DE 25 A 34  \
SEXO GRUPO PERMISOS ANYO_EXPEDICION                                     
M    A              2000                    0         0.0       0.000   
                    2001                    0         0.0       0.000   
                    2005                    0         0.0       0.000   
                    2006                    0         0.0       0.000   
                    2007                    0         0.0       0.000   
                    2009                    0         0.0       0.000   
                    2010                    0         0.0       0.000   
                    2011                    0         0.0       0.000   
                    2012                    0         0.0       0.000   
                    2013                    0         0.0       0.000   
                    2014                    0         0.0       3.000   
                    2015                    0         0.0       0.000   
                    2016                    0         0.0       2.000   
                    2017                    0         0.0      11.000   
                    2018                    0         1.0       9.000   
                    2019                    0         0.0       2.000   
     A1             2000                    0         0.0       0.000   
                    2001                    0         0.0       0.000   
                    2002                    0         0.0      26.000   
                    2003                    0         0.0      50.000   
                    2004                    0         0.0      72.000   
                    2005                    0         0.0      64.000   
                    2006                    0         0.0      90.000   
                    2007                    0         0.0      66.000   
                    2008                    0         0.0      72.000   
                    2009                    0         0.0      52.000   
                    2010                    0         0.0      37.000   
                    2011                    0         0.0      27.000   
                    2012                    0        14.0      23.000   
                    2013                    0        28.0       2.000   
...                                       ...         ...         ...   
V    D1             2008                    0         0.0       0.000   
                    2009                    0         0.0       0.000   
                    2010                    0         0.0       0.000   
                    2015                    0         0.0       1.000   
                    2016                    0         0.0       0.000   
                    2017                    0         0.0       0.000   
                    2018                    0         2.0       1.000   
     LVA            2000                    0         0.0       0.000   
                    2015                    0         0.0       0.000   
                    2018                    1         0.0       0.000   
     Otros          2000                    0         0.0       0.000   
                    2001                    0         0.0       0.000   
                    2002                    0         0.0     130.000   
                    2003                    0         0.0     561.000   
                    2004                    0         0.0     776.000   
                    2005                    0         0.0       1.060   
                    2006                    0         0.0       1.284   
                    2007                    0         0.0       1.532   
                    2008                    0         0.0       1.783   
                    2009                    0         0.0       1.385   
                    2010                    0         0.0       1.343   
                    2011                    0         0.0       1.099   
                    20

In [18]:
permisosOut = open ('data/permisosMadridTratado.csv','w')
i=0
with open ('data/permisosMadrid.csv',encoding="ISO-8859-1") as permisosIn:
        reader = csv.DictReader(permisosIn)
        for row in reader:
            #print ('grupo permisos {}'.format(row['GRUPO PERMISOS']))
            i += 1
            if i == 1:
                fieldnames = [*row]
                writer = csv.DictWriter(permisosOut,fieldnames)
                writer.writeheader()
            else
                acumularContadores (rowPrev, row)

            rowPrev = row
            
            for permiso in row['GRUPO PERMISOS'].split(','):
                if not '+E' in permiso:
                    row['GRUPO PERMISOS']=permiso
                    writer.writerow(rowOut)

permisosOut.close()
permisosIn.close()